In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from datetime import date
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
import re

In [3]:
df_movie = pd.read_parquet("ignore\df_movie.parquet")
pd.set_option('display.max_columns', None)
df_movie.head(3)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\manon\AppData\Local\Temp\ipykernel_11904\2570935709.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_movie = pd.read_parquet("ignore\df_movie.parquet")


,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_1_mean_movie,actor_2_mean_movie,actor_3_mean_movie,actor_4_mean_movie,actor_5_mean_movie,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5
0,11224,en,79.005,1970,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",79.00,62.53,62.53,79.00,79.00,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626
1,389,en,58.940,1970,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",27.72,27.40,44.92,58.94,58.94,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698
2,6844,en,56.194,1970,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",29.45,38.47,56.19,56.19,56.19,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539


In [4]:
numeric_cols = ['popularity_movie', 'vote_average', 'vote_count',
       'gender_director', 'popularity_director', 'birthday_director', 'actor_2_mean_movie', 'actor_3_mean_movie',
       'actor_4_mean_movie', 'actor_5_mean_movie', 'actor_gender_1',
       'actor_gender_2', 'actor_gender_3', 'actor_gender_4', 'actor_gender_5',
       'actor_popularity_1', 'actor_popularity_2', 'actor_popularity_3',
       'actor_popularity_4', 'actor_popularity_5']


categorical_cols = ['original_language', 'name_director', 'place_of_birth_director']

binarizer_cols = ['genre']

date_cols = ['release_date', 'birthday_director', 'deathday_director']

In [5]:
# Custom transformer for MultiLabelBinarizer
class MultiLabelBinarizerPipelineFriendly(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_

In [6]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

date_transformer = Pipeline(steps=[
    ('date', OrdinalEncoder())
])

# On combine tout dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('genres', MultiLabelBinarizerPipelineFriendly(), 'genre'),
        ('date', date_transformer, date_cols)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [7]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['popularity_movie',
                                                   'vote_average', 'vote_count',
                                                   'gender_director',
                                                   'popularity_director',
                                                   'birthday_director',
                                                   'actor_2_mean_movie',
                                                   'actor_3_mean_movie',
                                                   'actor_4_mean_movie',
                                                   'actor_5_mean_movie',
                                                   'actor_gender_1',
                                                   'actor_gender_2',
                                                   'actor_...
                                                   'actor_popularity_4',
                                                   'actor_popularity_5']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['original_language',
                                                   'name_director',
                                                   'place_of_birth_director']),
                                                 ('genres',
                                                  MultiLabelBinarizerPipelineFriendly(),
                                                  'genre'),
                                                 ('date',
                                                  Pipeline(steps=[('date',
                                                                   OrdinalEncoder())]),
                                                  ['release_date',
                                                   'birthday_director',
                                                   'deathday_director'])]))])

In [8]:
# Transformation des données
processed_data = pipeline.fit_transform(df_movie)

# Affichage des données transformées
transformed_df = pd.DataFrame(processed_data.todense(), columns=pipeline["preprocessor"].get_feature_names_out())

In [9]:
processed_data.todense().shape

(5771, 3863)

In [10]:
# Initialisation du modèle KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='manhattan')  # 5 voisins les plus proches hormis le point de référence lui meme
model = knn_model.fit(processed_data)

In [11]:
print(model)

NearestNeighbors(metric='manhattan', n_neighbors=6)


In [136]:
# Exemple de recommandation pour un item
def recommend_similar_films(title, 
                              data: pd.DataFrame=processed_data, 
                              model: NearestNeighbors=knn_model, 
                              original_data: pd.DataFrame=df_movie, 
                              n_neighbors: int=6) -> tuple[list[float], list[float]]:
    """
    Trouve les n éléments les plus proches pour un élément donné.

    Args:
        title: le titre d'un film
        data: Données transformées utilisées pour KNN = df processed data
        model: Modèle KNN pré-entraîné.
        original_data: Données originales (pour affichage) = df movies
        n_neighbors: Nombre de voisins à recommander = 6 pour avoir 5 recommandations

    Returns:
        films similaires: Indices et distances des filmes similaires.
    """
    index = original_data[original_data['title'] == title].index[0] # l'indice du filme recherché 
    distances, indices = model.kneighbors(data[index], n_neighbors=n_neighbors) 
    
    indices = indices[0][1:]
    id_movie = [int(original_data.loc[i, 'id_movie']) for i in indices] 
    return id_movie



In [137]:
recommend_similar_films('Dune')

c:\Users\sulta\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\sulta\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[985, 9426, 709, 2756, 638]